In [1]:
import pickle as pkl
# import pandas as pd
# from backtest_lib.examples.get_sp500_historical import fetch_history
# import yfinance as yf
# import time

# tickers = [line.rstrip("\n") for line in open("sp500_constituents.txt", "r").readlines()]


# history_batches = fetch_history(tickers, start="2020-01-01")
# pkl.dump(history_batches, open("./sp500.pkl", "wb"))
# history_batches = pkl.load(open("./sp500.pkl", "rb"))

# history = pd.concat(history_batches, axis=1)

# close_df = history.xs("Close", axis=1, level=1)

# open_df = history.xs("Open", axis=1, level=1)
# high_df = history.xs("High", axis=1, level=1)
# low_df = history.xs("Low", axis=1, level=1)
# vol_df = history.xs("Volume", axis=1, level=1)

# pkl.dump(close_df, open("./sp500_close.pkl", "wb"))

In [2]:
close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

In [3]:
# Convert to polars for your pipeline
import polars as pl

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns

In [4]:
pastview = close_pl.transpose()

pastview = pastview.rename(
    {orig: orig.replace("column_", "period_") for orig in pastview.columns}
)

window_size = 4
stagger = 0

pastview.select(pastview.columns[stagger : window_size + stagger])

period_0,period_1,period_2,period_3
f64,f64,f64,f64
85.949997,84.57,84.82,85.080002
94.900497,93.748497,95.143997,95.343002
86.949997,85.889999,86.339996,85.860001
75.087502,74.357498,74.949997,74.597504
46.110001,46.02,45.66,45.110001
…,…,…,…
80.139999,81.550003,82.790001,82.25
142.720001,142.100006,141.850006,140.25
59.200001,58.509998,57.709999,57.16


In [5]:
# now with the lib
from backtest_lib.market import PastView
from backtest_lib.market.polars_impl import PolarsPastView

close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_dataframe(close_prices_df)
print(isinstance(past_cost_prices, PastView))
print(past_cost_prices.by_period[-1]["AAPL"])

True
278.8500061035156


In [6]:
print(past_cost_prices.by_security["AAPL"])

PolarsTimeseries(_vec=shape: (1_486,)
Series: 'AAPL' [f64]
[
	75.087502
	74.357498
	74.949997
	74.597504
	75.797501
	…
	271.48999
	275.920013
	276.970001
	277.549988
	278.850006
], _axis=PeriodAxis(dt64=array(['2020-01-02T00:00:00.000000', '2020-01-03T00:00:00.000000',
       '2020-01-06T00:00:00.000000', ..., '2025-11-25T00:00:00.000000',
       '2025-11-26T00:00:00.000000', '2025-11-28T00:00:00.000000'],
      shape=(1486,), dtype='datetime64[us]'), labels=('2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26', '202

In [7]:
from backtest_lib.market import ByPeriod, BySecurity
from backtest_lib.market.polars_impl import PolarsByPeriod

print(
    isinstance(past_cost_prices.by_security, BySecurity),
    isinstance(past_cost_prices.by_period, ByPeriod),
    isinstance(past_cost_prices.by_period, PolarsByPeriod),
    isinstance(PolarsByPeriod, ByPeriod),
)


prices = past_cost_prices.by_period[-1]
prices_plus_one = past_cost_prices.by_period[-1] + 1
assert (prices_plus_one - prices).sum() == len(prices)
assert past_cost_prices.by_period[-1]._scalar_type is float
assert past_cost_prices.by_period[-1].floor()._scalar_type is int
assert (past_cost_prices.by_period[-1].floor() + 5 + 2.5 + 2)._scalar_type is float
assert (past_cost_prices.by_period[-1].floor() + 5 + 2 + 2)._scalar_type is int

True True True False


In [8]:
assert (
    past_cost_prices.by_security[["MSFT", "AAPL"]]
    .by_period[2:4]
    .by_period.as_df()
    .equals(
        past_cost_prices.by_period[2:4].by_security[["MSFT", "AAPL"]].by_period.as_df()
    )
)

In [9]:
import random
from time import perf_counter

import numpy as np

from backtest_lib.market.polars_impl import SeriesUniverseMapping
from backtest_lib.portfolio import QuantityPortfolio

secs = tuple(past_cost_prices.by_period[-1].keys())
qtys = [random.randint(0, 10) for sec in secs]

holdings = SeriesUniverseMapping.from_vectors(secs, qtys)

pf = QuantityPortfolio(cash=0, holdings=holdings)

prices = past_cost_prices.by_period[-1]

wpf = pf.into_weighted(prices)
t0 = perf_counter()
for _ in range(1000):
    wpf = pf.into_weighted(prices)
print(f"{perf_counter() - t0}")

0.024663996999152005


In [10]:
from backtest_lib.backtest import Backtest, BacktestSettings
from backtest_lib.market.polars_impl import SeriesUniverseMapping
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import (
    Decision,
    MarketView,
)
from backtest_lib.strategy.context import StrategyContext
from backtest_lib.universe import PastUniversePrices, Universe

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices), periods=dates
)
securities = tuple(securities)
universe = securities
initial_portfolio = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series(np.tile(1 / len(securities), len(securities)))
    ),
)

target_portfolio = initial_portfolio


def strategy(
    universe: Universe,
    market: MarketView,
    current_portfolio: WeightedPortfolio,
    ctx: StrategyContext,
):
    lookback_periods = 120
    if len(market.periods) < lookback_periods:
        return Decision(initial_portfolio)

    prices_120_periods_ago = market.prices.close.by_period[-(lookback_periods)]
    prices_now = market.prices.close.by_period[-1]
    momentum_mapping = (prices_now / prices_120_periods_ago) - 1
    stonks_with_posi_mom = [x for x, y in momentum_mapping.items() if y >= 0]
    stonks_with_neg_mom = [x for x, y in momentum_mapping.items() if y < 0]

    if len(stonks_with_neg_mom) == 0 or len(stonks_with_posi_mom) == 0:
        # do nothing
        return Decision(current_portfolio)

    momentum_weight_pos = 0.1 / len(stonks_with_posi_mom)
    momentum_weight_neg = 0.1 / len(stonks_with_neg_mom)

    portfolio_changes = {
        **{sec: momentum_weight_pos for sec in stonks_with_posi_mom},
        **{sec: -momentum_weight_neg for sec in stonks_with_neg_mom},
    }

    new_target_holdings = current_portfolio.holdings + portfolio_changes

    new_target_portfolio = WeightedPortfolio(
        current_portfolio.cash, new_target_holdings
    )

    return Decision(new_target_portfolio)


def index_strategy(
    universe: Universe,
    market: MarketView,
    current_portfolio: WeightedPortfolio,
    ctx: StrategyContext,
) -> Decision:
    return Decision(current_portfolio)


settings = BacktestSettings(allow_short=False)

bt = Backtest(
    strategy=index_strategy,
    universe=universe,
    market_view=market_view,
    initial_portfolio=initial_portfolio,
    settings=settings,
)


results = bt.run()

print(f"total return: {(results.total_return - 1) * 100:.2f}%")
end_port_df = pl.DataFrame(
    {
        "Security": bt._current_portfolio.holdings.keys(),
        "Weight": bt._current_portfolio.holdings.values(),
    }
)
print(f"end portfolio weights: {end_port_df}")
sorted_weights = sorted(
    bt._current_portfolio.holdings.items(), key=lambda x: x[1], reverse=True
)
print("sorted weights:", sorted_weights)

total return: 0.49%
end portfolio weights: shape: (488, 2)
┌──────────┬──────────┐
│ Security ┆ Weight   │
│ ---      ┆ ---      │
│ str      ┆ f64      │
╞══════════╪══════════╡
│ A        ┆ 0.001825 │
│ AMZN     ┆ 0.002512 │
│ ABT      ┆ 0.001515 │
│ AAPL     ┆ 0.003796 │
│ ADM      ┆ 0.001346 │
│ …        ┆ …        │
│ WELL     ┆ 0.002656 │
│ VMC      ┆ 0.002129 │
│ USB      ┆ 0.000847 │
│ V        ┆ 0.001789 │
│ TRV      ┆ 0.002177 │
└──────────┴──────────┘
sorted weights: [('NVDA', 0.030163459576339237), ('TSLA', 0.0153283937923521), ('SMCI', 0.014641688981654034), ('AVGO', 0.012775462098819253), ('PWR', 0.011563810183054376), ('CRWD', 0.010526210751337688), ('ANET', 0.010439147614473775), ('HWM', 0.00864562762272319), ('LLY', 0.008314386159576491), ('VST', 0.008004115658853164), ('AXON', 0.0072110859342268754), ('KLAC', 0.006580794602601173), ('MCK', 0.006493608942491278), ('EQT', 0.006004383757449691), ('LRCX', 0.005352601487889128), ('APH', 0.0052738193426140586), ('MPWR', 0.0

In [11]:
# for x, y in initial_portfolio.holdings.items():
#     print(f"{x},{y}")

rounded = [round(val, 6) for val in initial_portfolio.holdings.values()]

total_value = 1_000_000

quantities_with_1m = initial_portfolio.into_quantities(prices, total_value)
assert (
    quantities_with_1m.holdings * prices
).sum() + quantities_with_1m.cash == total_value

In [12]:
(initial_portfolio.holdings + {"AAPL": 0.1}).sum()

1.0999999999999994

In [13]:
past_cost_prices.by_period[0]

SeriesUniverseMapping(names=('A', 'AMZN', 'ABT', 'AAPL', 'ADM', 'AMP', 'T', 'AON', 'AIG', 'ADP', 'MMM', 'AJG', 'ABBV', 'ACGL', 'ANET', 'AXP', 'AES', 'AEP', 'AWK', 'ADBE', 'ALLE', 'APH', 'AMAT', 'APD', 'AMD', 'APTV', 'MO', 'AFL', 'APA', 'ARE', 'GOOGL', 'AKAM', 'ADI', 'GOOG', 'AIZ', 'AMT', 'AEE', 'AMGN', 'ALGN', 'ALL', 'AMCR', 'AOS', 'APO', 'ADSK', 'LNT', 'ATO', 'ACN', 'ALB', 'AME', 'CNC', 'CBRE', 'XYZ', 'TECH', 'AVB', 'BR', 'BX', 'BXP', 'AVGO', 'BA', 'CPB', 'CZR', 'BKR', 'BK', 'BF-B', 'CVX', 'CF', 'CDNS', 'BIIB', 'CNP', 'AXON', 'CPT', 'CHTR', 'CCL', 'AZO', 'BMY', 'CAT', 'SCHW', 'CRL', 'BRO', 'CDW', 'BG', 'KMX', 'CHRW', 'CBOE', 'BRK-B', 'COF', 'AVY', 'BLDR', 'BAX', 'BKNG', 'CAH', 'BLK', 'COR', 'BAC', 'BSX', 'BDX', 'BBY', 'BALL', 'DDOG', 'FSLR', 'DAL', 'CFG', 'CMI', 'CTAS', 'CLX', 'CB', 'CINF', 'CL', 'CVS', 'CCI', 'DG', 'CTVA', 'DVA', 'DHR', 'DECK', 'C', 'DLTR', 'KO', 'CPAY', 'CTSH', 'DRI', 'GLW', 'COP', 'COST', 'CSX', 'CMS', 'DELL', 'CMG', 'CME', 'DXCM', 'CSGP', 'CMCSA', 'CSCO', 'DAY', '

In [14]:
neg_series = bt._current_portfolio.holdings.as_series()
neg_mass = neg_series.clip(upper_bound=0).sum()
pos_mass = neg_series.clip(lower_bound=0).sum()
pos_only_series = neg_series.clip(lower_bound=0)

# l = []

# for val in pos_only_series:
#     l.append()

bt._current_portfolio.into_long_only()

WeightedPortfolio(cash=0.0, holdings=SeriesUniverseMapping(names=('A', 'AMZN', 'ABT', 'AAPL', 'ADM', 'AMP', 'T', 'AON', 'AIG', 'ADP', 'MMM', 'AJG', 'ABBV', 'ACGL', 'ANET', 'AXP', 'AES', 'AEP', 'AWK', 'ADBE', 'ALLE', 'APH', 'AMAT', 'APD', 'AMD', 'APTV', 'MO', 'AFL', 'APA', 'ARE', 'GOOGL', 'AKAM', 'ADI', 'GOOG', 'AIZ', 'AMT', 'AEE', 'AMGN', 'ALGN', 'ALL', 'AMCR', 'AOS', 'APO', 'ADSK', 'LNT', 'ATO', 'ACN', 'ALB', 'AME', 'CNC', 'CBRE', 'XYZ', 'TECH', 'AVB', 'BR', 'BX', 'BXP', 'AVGO', 'BA', 'CPB', 'CZR', 'BKR', 'BK', 'BF-B', 'CVX', 'CF', 'CDNS', 'BIIB', 'CNP', 'AXON', 'CPT', 'CHTR', 'CCL', 'AZO', 'BMY', 'CAT', 'SCHW', 'CRL', 'BRO', 'CDW', 'BG', 'KMX', 'CHRW', 'CBOE', 'BRK-B', 'COF', 'AVY', 'BLDR', 'BAX', 'BKNG', 'CAH', 'BLK', 'COR', 'BAC', 'BSX', 'BDX', 'BBY', 'BALL', 'DDOG', 'FSLR', 'DAL', 'CFG', 'CMI', 'CTAS', 'CLX', 'CB', 'CINF', 'CL', 'CVS', 'CCI', 'DG', 'CTVA', 'DVA', 'DHR', 'DECK', 'C', 'DLTR', 'KO', 'CPAY', 'CTSH', 'DRI', 'GLW', 'COP', 'COST', 'CSX', 'CMS', 'DELL', 'CMG', 'CME', 'DXC

In [15]:
close_df

Ticker,A,AMZN,ABT,AAPL,ADM,AMP,T,AON,AIG,ADP,...,ZTS,UNH,VRTX,VTR,DIS,WELL,VMC,USB,V,TRV
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,85.949997,94.900497,86.949997,75.087502,46.110001,169.229996,29.350452,208.789993,51.759998,170.320007,...,134.139999,292.500000,219.449997,56.389999,148.199997,80.139999,142.720001,59.200001,191.119995,137.509995
2020-01-03,84.570000,93.748497,85.889999,74.357498,46.020000,167.119995,29.501511,207.970001,51.360001,169.960007,...,134.160004,289.540009,217.979996,57.070000,146.500000,81.550003,142.100006,58.509998,189.600006,137.020004
2020-01-06,84.820000,95.143997,86.339996,74.949997,45.660000,167.669998,29.531721,208.570007,51.400002,170.190002,...,133.130005,291.549988,224.029999,57.660000,145.649994,82.790001,141.850006,57.709999,189.190002,137.169998
2020-01-07,85.080002,95.343002,85.860001,74.597504,45.110001,165.649994,29.645016,206.800003,51.110001,168.130005,...,133.580002,289.790009,223.789993,57.180000,145.699997,82.250000,140.250000,57.160000,188.690002,135.160004
2020-01-08,85.919998,94.598503,86.209999,75.797501,44.610001,167.139999,29.735649,207.649994,51.709999,169.710007,...,133.289993,295.899994,231.089996,56.900002,145.399994,82.269997,141.369995,57.040001,191.919998,136.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-21,151.250000,220.690002,128.110001,271.489990,58.630001,447.559998,25.930000,346.510010,75.959999,253.119995,...,122.059998,319.970001,426.760010,79.239998,104.279999,199.929993,287.829987,47.779999,327.980011,290.070007
2025-11-24,153.600006,226.279999,127.190002,275.920013,58.650002,446.049988,25.620001,344.700012,75.110001,250.100006,...,122.870003,319.049988,424.000000,78.800003,101.940002,202.300003,287.190002,47.750000,329.299988,290.839996
2025-11-25,157.199997,229.669998,128.050003,276.970001,59.700001,452.820007,25.860001,350.959991,75.010002,256.309998,...,127.889999,326.279999,429.820007,79.440002,103.309998,204.589996,292.720001,48.910000,334.529999,293.700012


In [16]:
import altair as alt

alt.theme.enable("dark")

ThemeRegistry.enable('dark')

In [17]:
(
    results.asset_returns.by_security["AAPL"].plot.kde()
    + results.asset_returns.by_security["NVDA"].plot.kde(color="limegreen")
    + results.asset_returns.by_security["AMZN"].plot.kde(color="gold")
).properties(width=1000)

alt.LayerChart(...)

In [18]:
(
    results.holdings.by_security["NVDA"].plot.line(color="limegreen", smoothing=10)
    + results.holdings.by_security["AAPL"].plot.line(smoothing=10)
    + results.holdings.by_security["AMZN"].plot.line(color="gold", smoothing=10)
).properties(width=1000)

alt.LayerChart(...)

In [19]:
results.holdings.by_security["NVDA"].plot.line()

alt.Chart(...)

In [20]:
results.market.prices.close.by_security.as_df()

date,A,AMZN,ABT,AAPL,ADM,AMP,T,AON,AIG,ADP,MMM,AJG,ABBV,ACGL,ANET,AXP,AES,AEP,AWK,ADBE,ALLE,APH,AMAT,APD,AMD,APTV,MO,AFL,APA,ARE,GOOGL,AKAM,ADI,GOOG,AIZ,AMT,…,VLO,TRMB,XEL,VTRS,WRB,WFC,ULTA,URI,WBD,UBER,WMB,ZBRA,WYNN,WTW,UPS,VZ,WST,WMT,VRSN,YUM,GWW,WAB,VRSK,WY,VST,WAT,VICI,ZTS,UNH,VRTX,VTR,DIS,WELL,VMC,USB,V,TRV
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2020-01-02 00:00:00,85.949997,94.900497,86.949997,75.087502,46.110001,169.229996,29.350452,208.789993,51.759998,170.320007,150.501678,95.510002,89.550003,43.400002,12.795,125.849998,20.030001,93.459999,120.199997,334.429993,125.389999,27.307501,62.200001,231.139999,49.099998,96.309998,49.25,53.349998,25.360001,159.729996,68.433998,87.639999,120.43,68.3685,132.360001,228.5,…,95.129997,41.939999,62.380001,20.65,30.64889,53.75,254.550003,168.649994,32.220001,30.99,23.49,259.140015,143.600006,204.279999,116.790001,61.049999,151.740005,39.646667,196.729996,102.169998,344.709991,80.949997,151.119995,29.389999,22.84,235.059998,25.280001,134.139999,292.5,219.449997,56.389999,148.199997,80.139999,142.720001,59.200001,191.119995,137.509995
2020-01-03 00:00:00,84.57,93.748497,85.889999,74.357498,46.02,167.119995,29.501511,207.970001,51.360001,169.960007,149.205688,95.309998,88.699997,43.349998,12.553125,124.599998,19.799999,93.360001,120.980003,331.809998,123.839996,27.022499,61.209999,226.0,48.599998,94.099998,49.630001,52.98,25.690001,160.639999,68.075996,87.239998,118.309998,68.032997,132.990005,228.610001,…,91.519997,41.509998,62.68,20.200001,30.688889,53.419998,250.169998,165.710007,32.029999,31.370001,23.59,256.049988,141.470001,204.330002,116.720001,60.400002,152.139999,39.296665,200.880005,101.849998,340.660004,80.330002,152.589996,29.440001,23.08,231.009995,25.299999,134.160004,289.540009,217.979996,57.07,146.5,81.550003,142.100006,58.509998,189.600006,137.020004
2020-01-06 00:00:00,84.82,95.143997,86.339996,74.949997,45.66,167.669998,29.531721,208.570007,51.400002,170.190002,149.347824,95.75,89.400002,43.52,12.67875,124.059998,20.030001,93.669998,120.75,333.709991,123.610001,26.8825,59.889999,225.899994,48.389999,92.349998,49.830002,52.830002,25.639999,161.029999,69.890503,87.550003,116.919998,69.710503,133.110001,228.550003,…,91.529999,41.310001,62.59,20.870001,30.644444,53.099998,250.949997,165.380005,31.959999,31.58,23.9,258.01001,141.190002,204.580002,116.199997,60.27,152.220001,39.216667,202.740005,101.790001,337.48999,80.0,152.940002,29.379999,22.690001,228.880005,25.040001,133.130005,291.549988,224.029999,57.66,145.649994,82.790001,141.850006,57.709999,189.190002,137.169998
2020-01-07 00:00:00,85.080002,95.343002,85.860001,74.597504,45.110001,165.649994,29.645016,206.800003,51.110001,168.130005,148.745819,94.720001,88.889999,43.16,12.803125,123.410004,20.1,93.690002,120.0,333.390015,122.129997,27.0275,61.619999,226.860001,48.25,91.190002,49.57,52.330002,32.509998,155.759995,69.755501,90.199997,119.580002,69.667,132.210007,223.679993,…,92.730003,41.150002,62.459999,21.17,30.084444,52.66,253.089996,165.080002,32.07,32.810001,23.9,256.470001,141.860001,204.160004,116.0,59.599998,151.830002,38.853333,203.210007,101.970001,337.839996,79.970001,154.220001,29.190001,22.83,231.979996,24.940001,133.580002,289.790009,223.789993,57.18,145.699997,82.25,140.25,57.16,188.690002,135.160004
2020-01-08 00:00:00,85.919998,94.598503,86.209999,75.797501,44.610001,167.139999,29.735649,207.649994,51.709999,169.710007,151.028427,94.849998,89.519997,42.73,12.93625,125.540001,20.059999,93.410004,120.489998,337.869995,122.650002,27.092501,61.580002,228.089996,47.830002,93.089996,50.080002,52.490002,32.73,157.830002,70.251999,91.400002,120.660004,70.216003,132.710007,225.619995,…,93.089996,41.32,62.400002,21.219999,30.297777,52.82,258.0,1

In [21]:
results.holdings.by_security["AAPL"].as_series().min()
results.weights.by_security["AAPL"].as_series().min()

0.0020238038522319753

In [22]:
from backtest_lib.market.plotting import UniverseMappingPlotAccessor
from backtest_lib.market.polars_impl._plotting import SeriesUniverseMappingPlotAccessor
from backtest_lib.market.polars_impl._universe_mapping import SeriesUniverseMapping
from backtest_lib.universe.universe_mapping import UniverseMapping

issubclass(SeriesUniverseMappingPlotAccessor, UniverseMappingPlotAccessor)
(SeriesUniverseMapping, UniverseMapping)

(backtest_lib.market.polars_impl._universe_mapping.SeriesUniverseMapping,
 backtest_lib.universe.universe_mapping.UniverseMapping)

In [23]:
results.weights.by_period[-1].plot.bar(top=25)

alt.Chart(...)

In [24]:
results.holdings.by_period[-1].plot.stacked_bar(
    top=5, bar_label="2025-01-01"
) + results.holdings.by_period[-3].plot.stacked_bar(top=5, bar_label="2025-01-02")

alt.LayerChart(...)

In [25]:
results.holdings.by_period[:10].by_period._period_axis.dt64

array(['2020-01-02T00:00:00.000000', '2020-01-03T00:00:00.000000',
       '2020-01-06T00:00:00.000000', ..., '2025-11-25T00:00:00.000000',
       '2025-11-26T00:00:00.000000', '2025-11-28T00:00:00.000000'],
      shape=(1486,), dtype='datetime64[us]')

In [ ]:
results.holdings.by_security[["AAPL", "NVDA"]].by_period[-200:].by_period.plot.bar()

In [ ]:
results.holdings.by_security[["AAPL", "NVDA"]].plot.line(smoothing=15)

alt.Chart(...)

In [ ]:
results.weights.plot("bar").properties(width=1200, height=1000)